### SETUP INICIAL DO PROJETO

In [ ]:

#importação das bibliotecase e pacotes necessários para a análise

import json
import numpy as np
import os
import pandas as pd
import pandas_gbq as gbq
import re
import seaborn as sns
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.bigquery_storage import BigQueryReadClient
from google.oauth2 import service_account

# Carrega o .env: onde estão as credenciais do projeto/repositório
load_dotenv("/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/.env")

# Detectar ambiente: como eu estou usando wsl-ubuntu, no VS Code  -  Windows, estava dando conflitos de path
if os.name == "nt":  # se Windows
    credentials_path = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"
else:  # se WSL/Linux
    credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"

# Parâmetros injetados pelo Papermill ou definidos manualmente, caso não existam no ambiente
# Tables_to_process: lista de tabelas que serão processadas
# Output_dataset: nome do dataset onde os dados processados serão armazenados, neste caso, raw_data_cleaned
if 'tables_to_process' not in locals():
    tables_to_process = [
        "desafioadventureworks-446600.raw_data.humanresources-employee"       
    ]

if 'output_dataset' not in locals():
    output_dataset = "desafioadventureworks-446600.raw_data_cleaned"

# Configs do cliente BigQuery: input de project e location de acordo com dados no Bigquery
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=os.getenv("BIGQUERY_PROJECT"), location="us-central1")


In [ ]:
# Print com a tabela que vai ser processada nesse notebook

print("Tabelas a processar:", tables_to_process)

In [ ]:
# Nome do dataset no Bigquery com os dados brutos (.csv) extraídos pelo Meltano 
dataset_id = 'raw_data'
print(dataset_id)

# Lista de tabelas do dataset raw_data no Bigquery
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)

# Exploratory Data Analysis (EDA) e Data Cleaning

### Glossário dos dados:

df = dataframe  

bkp = backup (cópia)

doc = documentação / documentar decisões ou análises


In [ ]:
# Configuração para que o df exiba todas as colunas e todas as linhas completas, e também, exiba o formato numérico com 2 dígitos após a vírgula

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 10000)
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# Dicionário para armazenar os df processados
df_processados = {}

# Iteração das tabelas e armazenamento em df
for input_table in tables_to_process:
    print(f"Processando tabela: {input_table}")
    
    # Nome da tabela com substituição de '-' por '_'
    table_name = input_table.split(".")[-1].replace("-", "_")  
    
    # Ler os dados da tabela do BigQuery para um df
    print("Lendo os dados do BigQuery...")
    query = f"SELECT * FROM `{input_table}`"
    table_data = client.query(query).to_dataframe()
    
    # Armazenar o df no dicionário
    df_processados[table_name] = table_data
    print(f"Tabela {table_name} processada e armazenada com sucesso.")

# Print de validação
print("Todas as tabelas foram processadas com sucesso!")


In [ ]:
# Listar todas as variáveis criadas dinamicamente
for table_name in df_processados.keys():
    print(f"Variável criada: {table_name}")  

In [ ]:
# Atribuir o df a uma variável com nome mais simples
humanresources_employee = df_processados['humanresources_employee']

print(f"Colunas: {humanresources_employee.shape[1]}\nLinhas: {humanresources_employee.shape[0]}")

In [ ]:
# Identificar duplicatas com base em 'businessentityid'
duplicatas = humanresources_employee[
    humanresources_employee.duplicated(subset=['businessentityid'], keep=False)
]

# Verificar se existem duplicatas
if not duplicatas.empty:
    # Ordenar duplicatas por 'businessentityid' e 'modifieddate'
    duplicatas_ordenadas = duplicatas.sort_values(by=['businessentityid', 'modifieddate'])

    # Exibir duplicatas ordenadas
    print("Duplicatas ordenadas:")
    print(duplicatas_ordenadas)
else:
    print("Não foram encontradas duplicatas.")

In [ ]:
# Remover duplicatas mantendo a última ocorrência com base em 'modifieddate', pois ela que indica a data da última modificação nos dados
# Importante, pois se houver erro na ingestão (duplicação), mantém os dados integros.

humanresources_employee = humanresources_employee.drop_duplicates(subset=['businessentityid'], keep='last')

print(f"Linhas após remover duplicatas (baseando-se na última 'modifieddate'): {len(humanresources_employee)}")

#bkp dos dados brutos
raw_data_bkp_2_sem_duplicatas = humanresources_employee.copy()


In [ ]:
# Ordenar e exibir o df por 'businessentityid'
humanresources_employee = humanresources_employee.sort_values(by=['businessentityid'])

print(humanresources_employee)

In [ ]:
#Verificar se as colunas de DATAS têm valores nulos (NaN em pandas), pois não pode object, para facilitar manipulação dos dados
print(humanresources_employee.info())

In [ ]:
# Iterar por todas as colunas do df, para verificar valores ausentes

# Verificar valores ausentes na coluna
for column in humanresources_employee.columns:   
    missing_rows = humanresources_employee[humanresources_employee[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
# Mostrar as primeiras linhas ausentes, se preciso for, limitar o head() para dar menos outputs ou limitar os outputs
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")

In [ ]:
# Valores únicos por coluna, para verificar se colunas como flags, normalmente booleanas, possuem apenas 1 ou 2 valores.

valores_unicos = humanresources_employee.nunique(dropna=False)

print("Valores únicos incluindo NaN:")
print(valores_unicos)

#doc: currentflag possue somente 1 valor, o que indica que pode ser somente valores True ou False.

In [ ]:
# Padronizar textos em title (ex: Production Supervisor) ou upper (ex:M)
humanresources_employee['jobtitle'] = humanresources_employee['jobtitle'].str.strip().str.upper()
humanresources_employee['gender'] = humanresources_employee['gender'].str.strip().str.upper()
humanresources_employee['maritalstatus'] = humanresources_employee['maritalstatus'].str.strip().str.upper()
humanresources_employee['loginid'] = humanresources_employee['loginid'].str.strip().str.upper()
humanresources_employee['rowguid'] = humanresources_employee['rowguid'].str.strip().str.upper()

print(humanresources_employee.head())


#doc: padronizar as strings nessa etapa, contribui para a execução das demais etapas do pipeline

In [ ]:
# Listar colunas binárias esperadas e verificar valores únicos e suas distribuições
binary_columns = ['currentflag', 'salariedflag']

for col in binary_columns:
    unique_values = humanresources_employee[col].unique()
    print(f"Valores únicos em '{col}': {unique_values}")
    print(f"Distribuição de '{col}':")
    print(humanresources_employee[col].value_counts())
    print()


#doc: garantir que colunas binárias contenham apenas valores esperados, no caso True ou False e identificar anomalias verificando os valores únicos em cada coluna; valores fora do padrão binário, facilitando a validação e correção
#doc: currentflag só tem True, isso indica que a coluna é constante e não agrega informações úteis para a análise, pois não há variabilidade nos dados, ao contrario de salariedflag, que é uma coluna válida para análise, já que possui variabilidade e pode ser utilizada para segmentar os dados

In [ ]:
# Verificar se há funcionários ativos com `false`*
print("Funcionários ativos errados:", humanresources_employee[humanresources_employee['currentflag'] != True])

# Validar datas
# Converter 'hiredate' e 'modifieddate' para datetime sem timezone
hiredate = pd.to_datetime(humanresources_employee['hiredate'], errors='coerce').dt.tz_localize(None)
modifieddate = pd.to_datetime(humanresources_employee['modifieddate'], errors='coerce').dt.tz_localize(None)

# Contratações futuras**
print("Contratações futuras:", humanresources_employee[hiredate > pd.Timestamp.now()])

# Modifieddate antes de hiredate***
print("Modifieddate antes de hiredate:", humanresources_employee[modifieddate < hiredate])


#doc*: funcionários desligados / demitidos (False) não devem ser tratados como ativos no sistema, evitando inconsistências
#doc**:datas futuras indicam erros nos registros, pois contratações devem ocorrer em datas passadas
#doc***: um registro não pode ser modificado antes da contratação, pois seria um erro lógico nos dados

In [ ]:
# Atualizar o dicionário df_processados com o df ajustado
df_processados['humanresources_employee'] = humanresources_employee

In [ ]:
# Estatísticas descritivas para verificar se ainda há o que ser feito antes de exportar os dados ao BigQuery

# Identificar colunas numéricas para análise de outliers
numeric_columns = ['sickleavehours', 'vacationhours']

# Estatísticas Descritivas das colunas numéricas*
print(humanresources_employee[numeric_columns].describe())

# Cálculo de limites para outliers (IQR)**
for col in numeric_columns:
    q1 = humanresources_employee[col].quantile(0.25)
    q3 = humanresources_employee[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Exibir os limites
    print(f"\nColuna: {col}")
    print(f"Limite inferior: {lower_bound}, Limite superior: {upper_bound}")
    
    # Detecção e Análise de Outliers***
    outliers = humanresources_employee[(humanresources_employee[col] < lower_bound) | (humanresources_employee[col] > upper_bound)]
    print(f"Outliers detectados ({len(outliers)}):")
    print(outliers[[col]])


#doc: as colunas analisadas não apresentam outliers, pois os dados estão dentro dos limites esperados, sugerindo que não há necessidade de tratamento adicional para valores extremos. Isso indica boa qualidade dos dados para essas variáveis e que elas estão prontas para serem exportadas ou utilizadas em análises e modelos
#doc*: realizar estatísticas descritivas para entender a centralidade e variação dos dados (valores médios, mínimos, máximos, etc.)
#doc**: calcular limites para identificar outliers (valores extremos que podem indicar erros ou casos excepcionais nos dados)
#doc***: verificar a existência de outliers para decidir ações como remoção, substituição ou tratamento, garantindo qualidade dos dados

In [ ]:

# Garantir que apenas tabelas únicas sejam exportadas
unique_df_processados = {k: v for k, v in df_processados.items()}

# Exportar tabelas para o BigQuery
for table_name, df_cleaned in unique_df_processados.items():
    # Nome da tabela no BigQuery
    output_table = f"{output_dataset}.{table_name}"

    # Configurar job de exportação
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  # Substituir dados existentes
    )
    
    # Exportar DataFrame para o BigQuery
    job = client.load_table_from_dataframe(df_cleaned, output_table, job_config=job_config)
    job.result()

    print(f"Tabela {table_name} exportada com sucesso para {output_table}.")